In [1]:
from pynq import Overlay
import struct
import time

# Загружаем overlay
overlay = Overlay("/home/xilinx/pynq/overlays/matrix_inverse/mat_inv.bit")
ip = overlay.matrix_inverse_0

# АДРЕСА ИЗ .hwh файла (относительные к базовому адресу)
CTRL_REG = 0x00
A_0 = 0x10      # A[0][0] и A[0][1]
A_1 = 0x18      # A[1][0] и A[1][1]  
INV_A_0 = 0x20  # inv_A[0][0] и inv_A[0][1]
INV_A_1 = 0x28  # inv_A[1][0] и inv_A[1][1]
SUCCESS = 0x30

def uint32_to_float(i):
    """Конвертируем uint32 в float"""
    return struct.unpack('f', struct.pack('I', i))[0]

def float_to_uint32(f):
    """Конвертируем float в uint32"""
    return struct.unpack('I', struct.pack('f', f))[0]

def write_matrix():
    """Записываем матрицу A в IP"""
    # Матрица A = [[4.0, 7.0], [2.0, 6.0]]
    ip.write(A_0, float_to_uint32(4.0))   # A[0][0]
    ip.write(A_0 + 4, float_to_uint32(7.0))  # A[0][1] 
    ip.write(A_1, float_to_uint32(2.0))   # A[1][0]
    ip.write(A_1 + 4, float_to_uint32(6.0))  # A[1][1]
    print("Матрица A записана в IP")

def read_results():
    """Читаем результаты из IP"""
    # В PYNQ read() возвращает int (32-bit значение)
    inv_00 = uint32_to_float(ip.read(INV_A_0))      # inv_A[0][0]
    inv_01 = uint32_to_float(ip.read(INV_A_0 + 4))  # inv_A[0][1]
    inv_10 = uint32_to_float(ip.read(INV_A_1))      # inv_A[1][0] 
    inv_11 = uint32_to_float(ip.read(INV_A_1 + 4))  # inv_A[1][1]
    
    success = ip.read(SUCCESS) & 0x1  # берем только младший бит
    
    print("\nРЕЗУЛЬТАТЫ:")
    print(f"inv_A[0][0] = {inv_00:.6f}")
    print(f"inv_A[0][1] = {inv_01:.6f}") 
    print(f"inv_A[1][0] = {inv_10:.6f}")
    print(f"inv_A[1][1] = {inv_11:.6f}")
    print(f"Success flag = {success}")

def main():
    print("ЗАПУСК ОБРАЩЕНИЯ МАТРИЦЫ 2x2")
    
    # 1. Записываем матрицу
    write_matrix()
    
    # 2. Запускаем IP
    print("Запуск вычисления...")
    ip.write(CTRL_REG, 1)  # записываем просто число 1
    
    # 3. Ждем завершения
    time.sleep(0.01)
    
    # 4. Читаем результаты
    read_results()
    
    print("\nВЫПОЛНЕНО!")

if __name__ == "__main__":
    main()

ЗАПУСК ОБРАЩЕНИЯ МАТРИЦЫ 2x2
Матрица A записана в IP
Запуск вычисления...

РЕЗУЛЬТАТЫ:
inv_A[0][0] = 0.600000
inv_A[0][1] = -0.700000
inv_A[1][0] = -0.200000
inv_A[1][1] = 0.400000
Success flag = 1

ВЫПОЛНЕНО!


In [5]:
from pynq import Overlay
import struct
import time
import numpy as np

# Загружаем overlay
overlay = Overlay("/home/xilinx/pynq/overlays/matrix_inverse/mat_inv.bit")
ip = overlay.matrix_inverse_0

# АДРЕСА ИЗ .hwh файла (относительные к базовому адресу)
CTRL_REG = 0x00
A_0 = 0x10      # A[0][0] и A[0][1]
A_1 = 0x18      # A[1][0] и A[1][1]  
INV_A_0 = 0x20  # inv_A[0][0] и inv_A[0][1]
INV_A_1 = 0x28  # inv_A[1][0] и inv_A[1][1]
SUCCESS = 0x30

def uint32_to_float(i):
    """Конвертируем uint32 в float"""
    return struct.unpack('f', struct.pack('I', i))[0]

def float_to_uint32(f):
    """Конвертируем float в uint32"""
    return struct.unpack('I', struct.pack('f', f))[0]

def write_matrix_to_ip(matrix):
    """Записываем матрицу 2x2 в IP"""
    ip.write(A_0, float_to_uint32(matrix[0][0]))   # A[0][0]
    ip.write(A_0 + 4, float_to_uint32(matrix[0][1]))  # A[0][1] 
    ip.write(A_1, float_to_uint32(matrix[1][0]))   # A[1][0]
    ip.write(A_1 + 4, float_to_uint32(matrix[1][1]))  # A[1][1]

def read_matrix_from_ip():
    """Читаем обратную матрицу из IP"""
    inv_00 = uint32_to_float(ip.read(INV_A_0))      # inv_A[0][0]
    inv_01 = uint32_to_float(ip.read(INV_A_0 + 4))  # inv_A[0][1]
    inv_10 = uint32_to_float(ip.read(INV_A_1))      # inv_A[1][0] 
    inv_11 = uint32_to_float(ip.read(INV_A_1 + 4))  # inv_A[1][1]
    success = ip.read(SUCCESS) & 0x1  # берем только младший бит
    
    return np.array([[inv_00, inv_01], [inv_10, inv_11]]), success

def compute_inverse(matrix):
    """Вычисляем обратную матрицу на ПЛИС"""
    write_matrix_to_ip(matrix)
    ip.write(CTRL_REG, 1)  # запускаем IP
    time.sleep(0.01)  # ждем завершения
    return read_matrix_from_ip()

def matrix_multiply(A, B):
    """Умножение матриц 2x2 (для проверки)"""
    return np.array([
        [A[0][0]*B[0][0] + A[0][1]*B[1][0], A[0][0]*B[0][1] + A[0][1]*B[1][1]],
        [A[1][0]*B[0][0] + A[1][1]*B[1][0], A[1][0]*B[0][1] + A[1][1]*B[1][1]]
    ])

def verify_inverse(A, inv_A, test_name):
    """Проверяем корректность обращения матрицы"""
    print(f"\n🔍 ПРОВЕРКА {test_name}:")
    
    # A × inv_A должно быть единичной матрицей
    I_calculated = matrix_multiply(A, inv_A)
    I_expected = np.eye(2)
    
    print(f"A × inv_A = ")
    print(f"[{I_calculated[0][0]:.4f}, {I_calculated[0][1]:.4f}]")
    print(f"[{I_calculated[1][0]:.4f}, {I_calculated[1][1]:.4f}]")
    
    # Проверяем близость к единичной матрице
    error = np.max(np.abs(I_calculated - I_expected))
    print(f"Максимальная ошибка: {error:.6f}")
    
    if error < 1e-4:
        print("✅ Проверка пройдена! A × inv_A ≈ I")
    else:
        print("❌ Проверка не пройдена!")

def test_case_1():
    """ТЕСТ 1: Оригинальная матрица [[4,7],[2,6]]"""
    print("=" * 50)
    print("🧪 ТЕСТ 1: Базовая матрица")
    print("Матрица A = [[4.0, 7.0], [2.0, 6.0]]")
    
    A = np.array([[4.0, 7.0], [2.0, 6.0]])
    inv_A, success = compute_inverse(A)
    
    print(f"\n📊 РЕЗУЛЬТАТ:")
    print(f"inv_A[0][0] = {inv_A[0][0]:.6f}")
    print(f"inv_A[0][1] = {inv_A[0][1]:.6f}") 
    print(f"inv_A[1][0] = {inv_A[1][0]:.6f}")
    print(f"inv_A[1][1] = {inv_A[1][1]:.6f}")
    print(f"Success flag = {success}")
    
    verify_inverse(A, inv_A, "ТЕСТ 1")

def test_case_2():
    """ТЕСТ 2: Единичная матрица (должна остаться единичной)"""
    print("\n" + "=" * 50)
    print("🧪 ТЕСТ 2: Единичная матрица")
    print("Матрица A = [[1.0, 0.0], [0.0, 1.0]]")
    
    A = np.eye(2)
    inv_A, success = compute_inverse(A)
    
    print(f"\n📊 РЕЗУЛЬТАТ:")
    print(f"inv_A[0][0] = {inv_A[0][0]:.6f}")
    print(f"inv_A[0][1] = {inv_A[0][1]:.6f}") 
    print(f"inv_A[1][0] = {inv_A[1][0]:.6f}")
    print(f"inv_A[1][1] = {inv_A[1][1]:.6f}")
    print(f"Success flag = {success}")
    
    verify_inverse(A, inv_A, "ТЕСТ 2")

def test_case_3():
    """ТЕСТ 3: Другая обратимая матрица"""
    print("\n" + "=" * 50)
    print("🧪 ТЕСТ 3: Матрица [[3.0, 1.0], [2.0, 4.0]]")
    print("Ожидаемый результат: [[0.4, -0.1], [-0.2, 0.3]]")
    
    A = np.array([[3.0, 1.0], [2.0, 4.0]])
    inv_A, success = compute_inverse(A)
    
    print(f"\n📊 РЕЗУЛЬТАТ:")
    print(f"inv_A[0][0] = {inv_A[0][0]:.6f}")
    print(f"inv_A[0][1] = {inv_A[0][1]:.6f}") 
    print(f"inv_A[1][0] = {inv_A[1][0]:.6f}")
    print(f"inv_A[1][1] = {inv_A[1][1]:.6f}")
    print(f"Success flag = {success}")
    
    verify_inverse(A, inv_A, "ТЕСТ 3")

def test_case_4():
    """ТЕСТ 4: Необратимая матрица (det = 0)"""
    print("\n" + "=" * 50)
    print("🧪 ТЕСТ 4: Необратимая матрица")
    print("Матрица A = [[1.0, 2.0], [2.0, 4.0]]")
    print("Ожидание: Success = 0 (неудача)")
    
    A = np.array([[1.0, 2.0], [2.0, 4.0]])  # det = 1*4 - 2*2 = 0
    inv_A, success = compute_inverse(A)
    
    print(f"\n📊 РЕЗУЛЬТАТ:")
    print(f"inv_A[0][0] = {inv_A[0][0]:.6f}")
    print(f"inv_A[0][1] = {inv_A[0][1]:.6f}") 
    print(f"inv_A[1][0] = {inv_A[1][0]:.6f}")
    print(f"inv_A[1][1] = {inv_A[1][1]:.6f}")
    print(f"Success flag = {success}")
    
    if success == 0:
        print("✅ Корректно обработана необратимая матрица!")
    else:
        print("❌ Ошибка: необратимая матрица была 'обращена'")

def test_case_5():
    """ТЕСТ 5: Матрица с отрицательными элементами"""
    print("\n" + "=" * 50)
    print("🧪 ТЕСТ 5: Матрица с отрицательными элементами")
    print("Матрица A = [[2.0, -1.0], [1.0, 3.0]]")
    
    A = np.array([[2.0, -1.0], [1.0, 3.0]])
    inv_A, success = compute_inverse(A)
    
    print(f"\n📊 РЕЗУЛЬТАТ:")
    print(f"inv_A[0][0] = {inv_A[0][0]:.6f}")
    print(f"inv_A[0][1] = {inv_A[0][1]:.6f}") 
    print(f"inv_A[1][0] = {inv_A[1][0]:.6f}")
    print(f"inv_A[1][1] = {inv_A[1][1]:.6f}")
    print(f"Success flag = {success}")
    
    verify_inverse(A, inv_A, "ТЕСТ 5")

def main():
    print("🚀 ЗАПУСК КОМПЛЕКСНЫХ ТЕСТОВ ОБРАЩЕНИЯ МАТРИЦ 2x2")
    print("=" * 50)
    
    # Запускаем все тесты
    test_case_1()  # Базовая матрица
    test_case_2()  # Единичная матрица  
    test_case_3()  # Другая обратимая матрица
    test_case_4()  # Необратимая матрица
    test_case_5()  # Матрица с отрицательными элементами
    
    print("\n" + "=" * 50)
    print("✅ ВСЕ ТЕСТЫ ВЫПОЛНЕНЫ!")

if __name__ == "__main__":
    main()

🚀 ЗАПУСК КОМПЛЕКСНЫХ ТЕСТОВ ОБРАЩЕНИЯ МАТРИЦ 2x2
🧪 ТЕСТ 1: Базовая матрица
Матрица A = [[4.0, 7.0], [2.0, 6.0]]

📊 РЕЗУЛЬТАТ:
inv_A[0][0] = 0.600000
inv_A[0][1] = -0.700000
inv_A[1][0] = -0.200000
inv_A[1][1] = 0.400000
Success flag = 1

🔍 ПРОВЕРКА ТЕСТ 1:
A × inv_A = 
[1.0000, 0.0000]
[0.0000, 1.0000]
Максимальная ошибка: 0.000000
✅ Проверка пройдена! A × inv_A ≈ I

🧪 ТЕСТ 2: Единичная матрица
Матрица A = [[1.0, 0.0], [0.0, 1.0]]

📊 РЕЗУЛЬТАТ:
inv_A[0][0] = 1.000000
inv_A[0][1] = -0.000000
inv_A[1][0] = -0.000000
inv_A[1][1] = 1.000000
Success flag = 1

🔍 ПРОВЕРКА ТЕСТ 2:
A × inv_A = 
[1.0000, 0.0000]
[0.0000, 1.0000]
Максимальная ошибка: 0.000000
✅ Проверка пройдена! A × inv_A ≈ I

🧪 ТЕСТ 3: Матрица [[3.0, 1.0], [2.0, 4.0]]
Ожидаемый результат: [[0.4, -0.1], [-0.2, 0.3]]

📊 РЕЗУЛЬТАТ:
inv_A[0][0] = 0.400000
inv_A[0][1] = -0.100000
inv_A[1][0] = -0.200000
inv_A[1][1] = 0.300000
Success flag = 1

🔍 ПРОВЕРКА ТЕСТ 3:
A × inv_A = 
[1.0000, 0.0000]
[0.0000, 1.0000]
Максимальная ошибка: 0.0